In [12]:
!nvidia-smi

Mon Dec  1 21:31:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.80                 Driver Version: 581.80         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   44C    P5             15W /   30W |    1831MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import torch
from torch import nn

# 获取设备
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:0')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=0))

In [14]:
# 查看GPU数量
torch.cuda.device_count()

1

In [15]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [16]:
# 查询张量所在设备
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [18]:
# 创建张量时指定设备
X = torch.ones(2, 3, device=try_gpu())
print(X)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [20]:
# 如果有多个GPU，可以在指定设备上创建随机张量
Y = torch.rand(2, 3, device=try_gpu(0))
print(Y)

tensor([[0.5862, 0.7700, 0.5845],
        [0.6297, 0.2006, 0.0305]], device='cuda:0')


In [21]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [23]:
Y + Z

tensor([[1.5862, 1.7700, 1.5845],
        [1.6297, 1.2006, 1.0305]], device='cuda:0')

In [24]:
Z.cuda(0) is Z

True

In [25]:
# 将模型参数复制到GPU
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [26]:
net(X)

tensor([[-0.2561],
        [-0.2561]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [27]:
# 查看模型参数所在设备
net[0].weight.data.device

device(type='cuda', index=0)